#### Installing and importing packages 

In [ ]:
pip install sntwitter

In [ ]:
import os
import snscrape.modules.twitter as sntwitter
import itertools
from langdetect import detect

#### Defining target location (longitude & latitude) and target radius so we can focus in the UK

In [1]:
loc = '52.976736061080295, -1.8943957535613414, 300km'

#### Getting keyword specific (toyota) tweets using snscrape and itertools

In [ ]:
df = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    'toyota geocode:"{}"'.format(loc)).get_items(), 10000))[['user', 'date','content']]

#### Extracting location, username, display name, date & time into separate columns

In [ ]:
df['user_location'] =  df['user'].apply(lambda x: x['location'])
df['user_handle'] =  df['user'].apply(lambda x: x['username'])

df['DISPLAY NAME'] =  df['user'].apply(lambda x: x['displayname'])


df['Dates'] = pd.to_datetime(df['date']).dt.date
df['Time'] = pd.to_datetime(df['date']).dt.time

df[['location1', 'location2', 'location3', 'location4', 'location5']] = df["user_location"].str.split(',', expand=True)

#### Detecting language of tweet

In [ ]:
for index, row in df['content'].iteritems():
    lang = detect(row) #detecting each row
    df.loc[index, 'language'] = lang

#### Removing tweets from the official Toyota page

In [ ]:
df2 = df[~df.user_handle.str.contains("toyota")]
df2 = df2[~df.user_handle.str.contains("Toyota")]

#### Saving output data

In [ ]:
os.chdir("datalocation")
df2.to_csv("toyota_tweets_UK.csv")